In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
ass = pd.read_csv("../cleaned data/AS.csv", low_memory=False, index_col=False)
mp = pd.read_csv("../cleaned data/MP.csv", low_memory=False, index_col=False)
up = pd.read_csv("../cleaned data/UP.csv", low_memory=False, index_col=False)
jh = pd.read_csv("../cleaned data/JH.csv", low_memory=False, index_col=False)
rj = pd.read_csv("../cleaned data/RJ.csv", low_memory=False, index_col=False)
bh = pd.read_csv("../cleaned data/BH.csv", low_memory=False, index_col=False)
cg = pd.read_csv("../cleaned data/CG.csv", low_memory=False, index_col=False)
od = pd.read_csv("../cleaned data/OD.csv", low_memory=False, index_col=False)
ut = pd.read_csv("../cleaned data/UT.csv", low_memory=False, index_col=False)

In [3]:
batch_size = 1800 
training_steps = 12000

In [4]:
result = pd.concat([ut, cg, jh, ass, od, rj, mp, bh, up])

In [5]:
result.head()

,state_code,record_code_iodine,sex,age,weight_in_kg,length_height_cm,haemoglobin_level,first_breast_feeding,is_cur_breast_feeding,day_or_month_for_breast_feeding,water_month,ani_milk_month,semisolid_month_or_day,solid_month,vegetables_month_or_day,illness_type
0,5.0,3,1,12.0,10.0,79.199997,11.000000,1.0,1.0,6.0,6.0,6.0,6.0,7.0,7.0,0.0
1,5.0,3,2,24.0,19.6,80.199997,7.366314,1.0,2.0,18.0,6.0,11.0,6.0,7.0,7.0,0.0
2,5.0,3,1,36.0,12.5,89.300003,8.275318,1.0,2.0,6.0,6.0,6.0,7.0,7.0,7.0,0.0
3,5.0,3,1,36.0,12.3,91.599998,10.000000,1.0,2.0,6.0,6.0,6.0,7.0,7.0,7.0,0.0
4,5.0,3,1,36.0,12.5,112.600000,3.000000,1.0,2.0,6.0,6.0,6.0,7.0,7.0,7.0,0.0


In [6]:
len(result.columns[:-1])

15

In [7]:
x_train, x_test, y_train, y_test = train_test_split(result.drop("illness_type", axis=1), result["illness_type"], test_size=0.3, random_state=22)

In [8]:
#Feature columns describe how to use the input.
my_feature_columns = []
for key in x_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

#Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # Two hidden layers of 10 nodes each.
        hidden_units=[15, 15 , 20],
        # Activation function 
        activation_fn = tf.nn.relu ,
        # The model must choose between 5 classes. 
        n_classes=5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_train_distribute': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6db7f69278>, '_service': None, '_model_dir': '/tmp/tmpjn423as_', '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_evaluation_master': '', '_save_summary_steps': 100, '_session_config': None, '_master': '', '_is_chief': True}


In [9]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

In [10]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [11]:
classifier.train(input_fn=lambda:train_input_fn(x_train, y_train.astype(int) , batch_size),steps= training_steps)
#y_train.value_counts()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpjn423as_/model.ckpt.
INFO:tensorflow:loss = 20252.715, step = 1
INFO:tensorflow:global_step/sec: 57.0416
INFO:tensorflow:loss = 1651.3523, step = 101 (1.754 sec)
INFO:tensorflow:global_step/sec: 59.25
INFO:tensorflow:loss = 1515.8413, step = 201 (1.688 sec)
INFO:tensorflow:global_step/sec: 59.6845
INFO:tensorflow:loss = 1594.6552, step = 301 (1.675 sec)
INFO:tensorflow:global_step/sec: 59.6698
INFO:tensorflow:loss = 1595.5105, step = 401 (1.676 sec)
INFO:tensorflow:global_step/sec: 59.2495
INFO:tensorflow:loss = 1563.7352, step = 501 (1.688 sec)
INFO:tensorflow:global_step/sec: 59.1206
INFO:tensorflow:loss = 1566.6228, step = 601 (1.691 sec)
INFO:tensorflow:global_step/sec: 59.3067
INFO:tensorflow:lo

INFO:tensorflow:global_step/sec: 59.5189
INFO:tensorflow:loss = 1477.7388, step = 8101 (1.680 sec)
INFO:tensorflow:global_step/sec: 59.8636
INFO:tensorflow:loss = 1365.7336, step = 8201 (1.670 sec)
INFO:tensorflow:global_step/sec: 59.9543
INFO:tensorflow:loss = 1320.7484, step = 8301 (1.668 sec)
INFO:tensorflow:global_step/sec: 59.5463
INFO:tensorflow:loss = 1358.9974, step = 8401 (1.679 sec)
INFO:tensorflow:global_step/sec: 59.551
INFO:tensorflow:loss = 1425.9854, step = 8501 (1.679 sec)
INFO:tensorflow:global_step/sec: 59.33
INFO:tensorflow:loss = 1418.2151, step = 8601 (1.686 sec)
INFO:tensorflow:global_step/sec: 59.7553
INFO:tensorflow:loss = 1399.1221, step = 8701 (1.673 sec)
INFO:tensorflow:global_step/sec: 59.7882
INFO:tensorflow:loss = 1453.214, step = 8801 (1.673 sec)
INFO:tensorflow:global_step/sec: 59.5968
INFO:tensorflow:loss = 1471.7429, step = 8901 (1.678 sec)
INFO:tensorflow:global_step/sec: 59.6078
INFO:tensorflow:loss = 1409.7853, step = 9001 (1.678 sec)
INFO:tensorflo

In [12]:
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(x_test, y_test.astype(int) , batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-20-15:31:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjn423as_/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-20-15:31:48
INFO:tensorflow:Saving dict for global step 12000: accuracy = 0.7408724, average_loss = 0.78905565, global_step = 12000, loss = 1363.601


In [14]:
print("Accuracy : " + str(eval_result['accuracy']) )

Accuracy : 0.7408724
